Perplexity 에이전트 시스템

In [3]:
# !pip install langchain_chroma pytube youtube-search youtbue-transcript-api next_asyncio wolframalpha arxiv

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

WolframAlpha API 활용을 위한 비동기 설정
- async 호출만 가능하므로 nest_asyncio이용

In [5]:
import asyncio
asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())

import nest_asyncio
nest_asyncio.apply()

In [6]:
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
wolfram = WolframAlphaAPIWrapper()

In [7]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
    focus: Literal["web", "academic", "video", "math"]

웹 검색, 눈문 검색, 수학 계산 도구 정의

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import YouTubeSearchTool
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
import re

web_tool = TavilySearchResults(max_results=3)

@tool
def academic_tool(query:str):
    """
    academic paper search tool
    """
    arxiv = ArxivAPIWrapper()
    docs = arxiv.run(query)
    return docs

@tool
def math_tool(query:str):
    """
    math tool
    """
    wolfram = WolframAlphaAPIWrapper()
    result = wolfram.run(query)
    return result

In [10]:
# Test
academic_tool.run("Deep Seek Principle")

"Published: 2023-06-04\nTitle: Efficient Exploration via Epistemic-Risk-Seeking Policy Optimization\nAuthors: Brendan O'Donoghue\nSummary: Exploration remains a key challenge in deep reinforcement learning (RL).\nOptimism in the face of uncertainty is a well-known heuristic with theoretical\nguarantees in the tabular setting, but how best to translate the principle to\ndeep reinforcement learning, which involves online stochastic gradients and\ndeep network function approximators, is not fully understood. In this paper we\npropose a new, differentiable optimistic objective that when optimized yields a\npolicy that provably explores efficiently, with guarantees even under function\napproximation. Our new objective is a zero-sum two-player game derived from\nendowing the agent with an epistemic-risk-seeking utility function, which\nconverts uncertainty into value and encourages the agent to explore uncertain\nstates. We show that the solution to this game minimizes an upper bound on the\

In [11]:
# Test
math_tool.run("123*123^2")

'Assumption: 123×123^2 \nAnswer: 1860867'

In [15]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools import YouTubeSearchTool
from langchain_community.document_loaders import YoutubeLoader
from langchain_core.documents import Document
import ast

youtube_search_tool = YouTubeSearchTool()

@tool
def video_tool(query: str) -> str:
    """
    Retriever tool for the transcript for a YouTube video.
    If user want to find some information, this tool is good to gather youtube video information.
    query should be given in string format.
    """
    urls = youtube_search_tool.run(query)
    urls = ast.literal_eval(urls)
    docs = []
    for url in urls:
        loader = YoutubeLoader.from_youtube_url(
            url, add_video_info=True, language=["en", "ko"]
        )
        scripts = loader.load()
        script_content = scripts[0].page_content
        title = scripts[0].metadata['title']
        author = scripts[0].metadata['author']
        doc = Document(page_content=script_content, metadata={"source": url, "title": title, "author":author})
        docs.append(doc)

    text_splitter = RecursiveCharacterTextSplitter(
        separators = ["\n\n", "\n", ".", ",", " ", ""],
        chunk_size = 1000, chunk_overlap=0
    )
    texts = text_splitter.split_documents(docs)
    embeddings = OpenAIEmbeddings()
    db = Chroma.from_documents(texts, embeddings)
    retriever = db.as_retriever()
    retrieved_docs = retriever.invoke(query)

    video_results = []
    for doc in retrieved_docs:
        title = doc.metadata.get('title', 'No title available')
        author = doc.metadata.get('author', 'No author available')
        script_content = doc.page_content

        video_info = f"""
        Video Information:
        ------------------
        Title: {title}
        Author: {author}
        Transcript: {script_content}
        ------------------
        """
        video_results.append(video_info)
    
    return "\n\n".join(video_results)

In [20]:
video_tool.run("테디노트가 누구야?")

PytubeError: Exception while accessing title of https://youtube.com/watch?v=slqdel6HoCI. Please file a bug report at https://github.com/pytube/pytube

In [ ]:

@tool
def youtube_tool(query:str):
    """
    youtube tool
    """
    yt = YouTubeSearchTool()
    result = yt.run(query)
    return result